In [ ]:
def train(data, save_path):
    """
    Train a neural network to predict buy/sell signals based on market data.

    Args:
        data (pd.DataFrame): Historical market data (including 'Adj Close').
        save_path (str): Path to save the trained neural network model.

    Returns:
        None
    """
    data = data.copy()

    # Prepare features and labels
    data['Return'] = data['Adj Close'].pct_change()
    data['Target'] = np.where(data['Return'].shift(-1) > 0, 1, 0)
    data.dropna(inplace=True)

    features = data[['Return']].values
    labels = data['Target'].values

    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Build a simple neural network
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

    # Save the trained model
    model.save(save_path)


In [ ]:
def optimize_nn(data, param_grid, save_path, epochs=50):
    """
    Optimize a neural network by tuning hyperparameters.

    Args:
        data (pd.DataFrame): Historical market data (including 'Adj Close').
        param_grid (dict): Grid of hyperparameters to test.
        save_path (str): Path to save the best neural network model.
        epochs (int): Number of training epochs.

    Returns:
        dict: Best hyperparameters and their performance.
    """
    data = data.copy()

    # Prepare features and labels
    data['Return'] = data['Adj Close'].pct_change()
    data['Target'] = np.where(data['Return'].shift(-1) > 0, 1, 0)
    data.dropna(inplace=True)

    features = data[['Return']].values
    labels = data['Target'].values

    # Standardize features
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Search for best hyperparameters
    best_performance = -float('inf')
    best_params = {}

    for params in param_grid:
        # Build the model with the current parameters
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(X_train.shape[1],)),
            tf.keras.layers.Dense(params['hidden_units'], activation='relu'),
            tf.keras.layers.Dropout(params['dropout']),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=params['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])

        # Train the model
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        model.fit(X_train, y_train, validation_data=(X_test, y_test),
                  epochs=epochs, batch_size=params['batch_size'], callbacks=[early_stopping], verbose=0)

        # Evaluate the model
        evaluation = model.evaluate(X_test, y_test, verbose=0)
        accuracy = evaluation[1]

        # Save the best model
        if accuracy > best_performance:
            best_performance = accuracy
            best_params = params
            model.save(save_path)

    print(f"Best Parameters: {best_params}, Performance: {best_performance}")
    return {"best_params": best_params, "performance": best_performance}


In [ ]:
ticker = "AAPL"
start_date = "2015-01-01"
end_date = "2023-12-31"
data = get_market_data(ticker, start_date, end_date)

# Train a neural network
model_path = "advanced_nn_model.h5"
train_results = train_nn_advanced(data, model_path)

# Optimize a neural network
param_grid = [
    {'hidden_units': 64, 'dropout': 0.2, 'batch_size': 32, 'optimizer': 'adam'},
    {'hidden_units': 128, 'dropout': 0.3, 'batch_size': 16, 'optimizer': 'adam'},
    {'hidden_units': 128, 'dropout': 0.3, 'batch_size': 32, 'optimizer': 'sgd'}
]
best_nn = optimize_nn(data, param_grid, "best_nn_model.h5")
